# Modules

In [101]:
import re
import pandas as pd
import matplotlib.pyplot as plt
import emojis

# Preprocessing

In [102]:
def istime(s):
    pattern = '^([0-9]+)/([0-9]+)/([0-9]+), ([0-9]+):([0-9]+)[ ]?' 
    result = re.match(pattern, s)
    if result:
        return True
    return False

def isNotif(m):
    if len(m.split(": ")) >=2:
        return True
    return False

def get_slice_data(s):
    Line = s.split(' - ') 
    dateTime = Line[0]
    date, time = dateTime.split(', ') 
    message = ' '.join(Line[1:])
    if isNotif(message):
        splitMessage = message.split(':') 
        name = splitMessage[0] 
        message = ' '.join(splitMessage[1:])
    else:
        name = "Notifications"
        

    return date, time, name, message


In [103]:
raw_data = open("test.txt","r")
raw_data = raw_data.read().replace("\u202f", " ").split("\n")

lindata= []
buff = ""

for lin in raw_data:
    if istime(lin):
        lindata.append(buff)
        buff = ""
    buff += lin
    #print(istime(lin))

lindata = lindata[1:]
#lindata

In [104]:
data=dict()
data["Date"] = []
data["Time"] = []
data["Name"] = []
data["Message"] = []

for lin in lindata:
    date, time, name, message = get_slice_data(lin)
    data["Date"].append(date)
    data["Time"].append(time)
    data["Name"].append(name)
    data["Message"].append(message)
#data
df = pd.DataFrame(data,columns=['Date', 'Time', 'Name', 'Message'])
df["Date"] = pd.to_datetime(df["Date"])
#df[100:200]
    